# 1.1: Pull tweets from Twitter API

Pull Old tweets based keywords, geo/location and time stamp

In [18]:
from TwitterAPI import TwitterAPI
import json

import pandas as pd
import time

In [23]:
# put your key here
consumer_key='zRflCqc3by5SDMy2HibYzUu4l'
consumer_secret='gBGHOP2LTQlSuaYM70BTTjzXrdJuIl3efzD4KHYmlwXJn9n32c'
access_token_key='53118685-gXVeteybKfgrkA0A0mWMM3Hv9Rj9qbz5N2Es6KwvZ'
access_token_secret='5KF8dvzvifCiRwKymDAduqn7DBDNyKTYkN4V0FitXp2UM'

In [24]:
# put your params here: change geo, time and keywords
params = {
    'query':'point_radius:[-121.6219 39.7520 25mi] (wildfire OR fire OR campfire)',
    'fromDate':'201811070000',
    'toDate':'201901310000'
}

In [25]:
# functions used to pull call API multiple times
# Please read all comments as I put below and make changes accordingly

api = TwitterAPI(consumer_key=consumer_key,
                 consumer_secret=consumer_secret,
                 access_token_key=access_token_key,
                 access_token_secret=access_token_secret)

def _get_tweet_count(found_tweets):
    return len(set(tweet['id'] for tweet in found_tweets))

def search(api, params, next_key=None):
    PRODUCT = 'fullarchive'
    LABEL = 'development' # This is specific to your application
               # i.e. whatever label you set for your Dev environment, and is case sensitive
    if next_key:
        params['next']= next_key
    r = api.request('tweets/search/%s/:%s' % (PRODUCT, LABEL), params)
    print(r.json().keys())
    try:
        results = r.json()['results']
    except:
        print(r.json())
        return None, None
    try:
        next_new = r.json()['next']
    except:
        return results, None
    return results, next_new

def search_tweets(api, params, counts):
#     results = []
    global results
    MAX_COUNTER = 20 # number of times running api
                     # this function only needs to be called once to get 2000 tweets
                     # since we only have 50 times, please be careful when using this function
                     # you can set MAX_COUNTER =1, counts=100 to test at first
                     
                     # BUT eg. if there are only 300 tweets based on pre-set params, it will only run 3 times
                     # so use advanced search in twitter to find the best location/time/keyword(best params) at first
                     # to ensure you can get more than 2000 tweets based on your params
                     # I think we don't need 4 keywords, we mainly need location/time of two or three big disasters
                     # If you want to call this function several times, be sure you are using different params!!!
                     # or it will give you duplicated tweets
                     # save the results each time when you run it, do not lose your chance to use twitter api
    query_counter = 0
    tweet_counter = 0
    next_key = ''
    while len(results) < counts and query_counter <= MAX_COUNTER:
        result, next_key = search(api, params, next_key)
        print('got %s results' % str(len(result)))
        print('Next: %s' % next_key)
        results.extend(result)
        query_counter += 1
        if _get_tweet_count(results) == tweet_counter:
            break
        if next_key == None:
            break
        tweet_counter = _get_tweet_count(results)
        time.sleep(2)
    return results


In [26]:
results = []
# used above functions
# 2000 is the number of the tweets we gonna get
test_results = search_tweets(api, params, 2000)

dict_keys(['results', 'next', 'requestParameters'])
got 100 results
Next: eyJhdXRoZW50aWNpdHkiOiI0ZTc5NDIwN2QxMzVhODhhYTgxMTM0YTdjOTM4NDRjODNiODJlZDUzYjNkZmQ1NTYyZTZmMzhjZDE2MjgyNGJlIiwiZnJvbURhdGUiOiIyMDE4MTEwNzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDEzMTAwMDAiLCJuZXh0IjoiMjAxOTAxMzEwMDAwMDAtMTA3MDkxODAzNzE0MDg4MTQxMy0wIn0=
dict_keys(['results', 'next', 'requestParameters'])
got 100 results
Next: eyJhdXRoZW50aWNpdHkiOiJjM2NmNmIwODIzOWU2MWM5ZWQ4YTlhOWZiYTc0NTIxZmFiNTdmYTIxMWQyOGJkNDcxN2U3ZDM4YTg3MTU5MDgyIiwiZnJvbURhdGUiOiIyMDE4MTEwNzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDEzMTAwMDAiLCJuZXh0IjoiMjAxOTAxMzEwMDAwMDAtMTA2NjExNTk2MjM0MzE3NDE0NC0wIn0=
dict_keys(['results', 'next', 'requestParameters'])
got 100 results
Next: eyJhdXRoZW50aWNpdHkiOiJlYTY5YTRhMjY2YzY4NzcwN2VkYTgzNzU3MDBhOTYxNGEwNDU3YWNhOGYzOTkxNTdiNjM4MzgxYWMwMmI0M2Y1IiwiZnJvbURhdGUiOiIyMDE4MTEwNzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDEzMTAwMDAiLCJuZXh0IjoiMjAxOTAxMzEwMDAwMDAtMTA2NDYwOTQ0OTI3OTcxNzM3Ni0wIn0=
dict_keys(['results', 'next', 'requestParameters'])
got

In [27]:
# Info pulled from tweets
USEFUL_INFO_KEYS = ['created_at', 'text', 'longitute', 'latitute', 'retweet_count', 'favorite_count', 'hashtags']

def extract_tweets(tweets):
    result = []
    for tweet in tweets:
        info = {}
        for key in USEFUL_INFO_KEYS:
            if key == 'hashtags':
                info[key] = [i['text'] for i in tweet['entities']['hashtags']]
            elif key == 'longitute':
                try:
                    info[key] = tweet['coordinates']['coordinates'][0]
                except TypeError:
                    pass
            elif key == 'latitute':
                try:
                    info[key] = tweet['coordinates']['coordinates'][1]
                except TypeError:
                    pass
            else:
                info[key] = tweet[key]
        result.append(info)
    return result

In [28]:
# convert results into dataframe
extracted_test = extract_tweets(test_results)
camp_fire_2 = pd.DataFrame(extracted_test)

In [29]:
# export to csv
camp_fire_2.to_csv('camp_fire_2.csv', index=False)

In [30]:
camp_fire_2

,created_at,favorite_count,hashtags,latitute,longitute,retweet_count,text
0,Wed Jan 30 19:35:24 +0000 2019,0,[],39.732610,-121.822170,0,jpmorgan chase and neighborworks is holding a ...
1,Tue Jan 29 18:20:56 +0000 2019,15,"[buttecollege, campfire, buttestrong]",39.649774,-121.644863,1,Laptops arrived! These are going to #buttecoll...
2,Tue Jan 29 13:23:08 +0000 2019,0,[],39.761700,-121.609000,0,"Hope in Paradise. \nThis is Josh, he is standi..."
3,Mon Jan 28 02:26:26 +0000 2019,0,[],39.529546,-121.473999,0,"Sunday evening by the fire. @ Oroville East, C..."
4,Sun Jan 27 07:20:29 +0000 2019,1,[],39.756978,-121.845589,0,"We’re headed to Chico, California tomorrow to ..."
5,Sun Jan 27 02:32:24 +0000 2019,1,"[GiveItUp, MichaelCapponi, GlobalEmpowermentMi...",39.761700,-121.609000,1,#GiveItUp for #MichaelCapponi's #GlobalEmpower...
6,Sat Jan 26 15:15:47 +0000 2019,3,[],39.761700,-121.609000,2,"Paradise, California: January 25, 2019 \nYeste..."
7,Thu Jan 24 00:49:22 +0000 2019,0,[],39.512900,-121.558300,0,Mod MOVERS is 30 minutes away from what’s refe...
8,Wed Jan 23 23:37:27 +0000 2019,0,[],39.728734,-121.839728,0,On today’s show we continue our updates on the...
9,Sat Jan 19 03:20:59 +0000 2019,1,[CampFire],39.761700,-121.609000,0,Such a joy to see a group of @challengecsuc st...
